In [5]:
import json
import random
import PIL

In [7]:
class FashionIQ():
  def __init__(self, path, split='train', transform=None):
    super(FashionIQ, self).__init__()
    
    self.split = split
    self.transform = transform
    self.img_path = path + '/'
    
    failures = [u'B00AZJJGDS', u'B00BTDROGU', u'B00AZJJER6', u'B00AZJJYBM', u'B009H3SDPK', u'B00AKQBDKU', u'B00AZJJE1C', u'B00BTDS338', u'B0082P4LLO', u'B00CM2RZ3E', u'B00C621JPK', u'B00AZGHX7M', u'B00A885VUI', u'B0057LRI6Q', u'B008583YKC', u'B009NY5YZA']

    data = {
        'image_splits': {},
        'captions': {}
    }
    import os
    for k in data:
        print(os.listdir(path + '/' + k))
        for f in os.listdir(path + '/' + k):
          print(f)
          if (split == 'train' and 'train' in f) or (split == 'test' and 'val' in f):
            d = json.load(open(path + '/' + k + '/' + f))
            data[k][f] = d

    imgs = []
    asin2id = {}
    for k in data['image_splits']:
        for asin in data['image_splits'][k]:
            if asin in failures:
                continue
            asin2id[asin] = len(imgs)
            imgs += [{
                'asin': asin,
                'file_path': path + '/images/' + asin + '.jpg',
                'captions': [asin2id[asin]]
            }]
    print('write attribute2idx.json')
    json_object = json.dumps(asin2id, indent = 4)
    # Writing to sample.json
    with open("attribute2idx.json", "w") as outfile:
        outfile.write(json_object)

    queries = []
    for k in data['captions']:
        for query in data['captions'][k]:
            if query['candidate'] in failures or query['target'] in failures:
                continue
            query['source_id'] = asin2id[query['candidate']]
            query['target_id'] = asin2id[query['target']]
            query['captions'] = [c.encode('utf-8') for c in query['captions']]
            queries += [query]
            
    
    self.data = data
    self.imgs = imgs
    self.queries = queries
    
    if split == 'test':
        self.test_queries = [{
              'source_img_id': query['source_id'],
              'target_img_id': query['target_id'],
              'target_caption': query['target_id'],
              'target_caption': query['target_id'],
              'mod': {'str': query['captions'][0] + ' inadditiontothat ' + query['captions'][1]}
          } for query in queries]

  def get_all_texts(self):
    texts = ['inadditiontothat']
    for query in self.queries:
        texts += query['captions']
    return texts

  def __len__(self):
    return len(self.imgs)

  def generate_random_query_target(self):
    query = random.choice(self.queries)
    mod_str = random.choice([
            query['captions'][0] + ' inadditiontothat ' + query['captions'][1],
            query['captions'][1] + ' inadditiontothat ' + query['captions'][0]
        ])
        
    return {
      'source_img_id': query['source_id'],
      'source_img_data': self.get_img(query['source_id']),
      'target_img_id': query['target_id'],
      'target_caption': query['target_id'],
      'target_img_data': self.get_img(query['target_id']),
      'target_caption': query['target_id'],
      'mod': {'str': mod_str}
    }

  def get_img(self, idx, raw_img=False):
    img_path = self.imgs[idx]['file_path']
    with open(img_path, 'rb') as f:
      img = PIL.Image.open(f)
      img = img.convert('RGB')
    if raw_img:
      return img
    if self.transform:
      img = self.transform(img)
    return img

In [8]:
path = '/home/piai/chan/largescale_multimedia/project/fashion-iq/data'
fashioniq = FashionIQ(path=path)

['split.toptee.train.json', 'split.dress.test.json', 'split.shirt.train.json', 'split.toptee.val.json', 'split.dress.train.json', 'split.shirt.val.json', 'split.dress.val.json', 'split.shirt.test.json', 'split.toptee.test.json']
split.toptee.train.json
split.dress.test.json
split.shirt.train.json
split.toptee.val.json
split.dress.train.json
split.shirt.val.json
split.dress.val.json
split.shirt.test.json
split.toptee.test.json
['cap.toptee.val.json', 'cap.toptee.test.json', 'cap.shirt.test.json', 'cap.shirt.val.json', 'cap.dress.val.json', 'cap.dress.test.json', 'cap.toptee.train.json', 'cap.dress.train.json', 'cap.shirt.train.json', 'dict.dress.json', 'dict.shirt.json', 'dict.toptee.json']
cap.toptee.val.json
cap.toptee.test.json
cap.shirt.test.json
cap.shirt.val.json
cap.dress.val.json
cap.dress.test.json
cap.toptee.train.json
cap.dress.train.json
cap.shirt.train.json
dict.dress.json
dict.shirt.json
dict.toptee.json
write attribute2idx.json


In [3]:
import json
import os

In [8]:
path = './resized_images/'
flist = ['train', 'val', 'test']
clothes = ['dress', 'toptee', 'shirt']

In [2]:
for f in flist:
    f = open('./image_splits/split.dress.train.json')
    
    # returns JSON object as
    # a dictionary
    dress_split = json.load(f)
    dress_split

['B008BHCT58',
 'B00CHSEGYE',
 'B00BI588U0',
 'B007BA3RAM',
 'B00BGYFWOI',
 'B00BZ8GPVO',
 'B00E36QG82',
 'B008KZR6WM',
 'B00684UYEE',
 'B00EV1B9C2',
 'B009LJ8LAM',
 'B008U8XOAC',
 'B000FSDAIG',
 'B00GXZYFZW',
 'B00CTSFYSE',
 'B008ULB0PK',
 'B00C97K8TK',
 'B00FX6NNVE',
 'B002GT074M',
 'B00C61ORU0',
 'B00EWDKINU',
 'B00BB7FN6C',
 'B009N9YUP0',
 'B00C6P5XNG',
 'B00EOSK2DY',
 'B0057WSW88',
 'B00DYXGVW6',
 'B004TM3KNE',
 'B00AIXOAW8',
 'B008BHHWLE',
 'B00A8F9ZJY',
 'B00A8FA3RW',
 'B0058VZDAS',
 'B007CVK41K',
 'B00G5N3J04',
 'B007EBPVAW',
 'B00CIGLQUC',
 'B00FJDYWAC',
 'B005F059JM',
 'B004NRNQRU',
 'B008UWJHFO',
 'B00DGP705O',
 'B007SRJ6HQ',
 'B00COYT3JE',
 'B0065RQ4LG',
 'B008R7871C',
 'B00438R7EM',
 'B001E8XULI',
 'B00CQ7LKS6',
 'B00A6FQH92',
 'B006WJ2LIW',
 'B00BL4U932',
 'B00E0WP2Y8',
 'B009DMFFLK',
 'B009S3EUM4',
 'B004IWRILU',
 'B003ILMX5I',
 'B00BX9FHJQ',
 'B008BRM2GY',
 'B00A8FAMPA',
 'B002Z7FPEM',
 'B00CHS6D22',
 'B005MGDYAA',
 'B0032FOQPA',
 'B0093UBZJ8',
 'B00B2JIGCC',
 'B004L92V

In [7]:
dress_path = path+'dress/'
for dress in dress_split:
    print(dress)
    fname = dress+'.png'
    os.system('mv %s/%s %s/%s'%(path, fname, dress_path, fname))

B008BHCT58
B00CHSEGYE
B00BI588U0
B007BA3RAM
B00BGYFWOI
B00BZ8GPVO
B00E36QG82
B008KZR6WM
B00684UYEE
B00EV1B9C2
B009LJ8LAM
B008U8XOAC
B000FSDAIG
B00GXZYFZW
B00CTSFYSE
B008ULB0PK
B00C97K8TK
B00FX6NNVE
B002GT074M
B00C61ORU0
B00EWDKINU
B00BB7FN6C
B009N9YUP0
B00C6P5XNG
B00EOSK2DY
B0057WSW88
B00DYXGVW6
B004TM3KNE
B00AIXOAW8
B008BHHWLE
B00A8F9ZJY
B00A8FA3RW
B0058VZDAS
B007CVK41K
B00G5N3J04
B007EBPVAW
B00CIGLQUC
B00FJDYWAC
B005F059JM
B004NRNQRU
B008UWJHFO
B00DGP705O
B007SRJ6HQ
B00COYT3JE
B0065RQ4LG
B008R7871C
B00438R7EM
B001E8XULI
B00CQ7LKS6
B00A6FQH92
B006WJ2LIW
B00BL4U932
B00E0WP2Y8
B009DMFFLK
B009S3EUM4
B004IWRILU
B003ILMX5I
B00BX9FHJQ
B008BRM2GY
B00A8FAMPA
B002Z7FPEM
B00CHS6D22
B005MGDYAA
B0032FOQPA
B0093UBZJ8
B00B2JIGCC
B004L92VKI
B008BPV244
B00APFED80
B0051T2TSQ
B00E4NM9LW
B008OWQHUS
B00509LC3U
B00ECWW762
B00E9A9DZU
B008BTS9VY
B00AYKTS5Y
B006ZQW628
B005SOW7JA
B004R1RFL0
B0096NM93I
B0071NUAY0
B001M1X8WI
B00CJ5O6QS
B00CN4XVGG
B003VS71VE
B009KP6QZO
B00940K6UG
B00CF1X8C4
B005HYUD42
B008ULB33O

In [ ]:
f = open('./image_splits/split.dress.train.json')
 
# returns JSON object as
# a dictionary
dress_split = json.load(f)
dress_split

In [ ]:
for toptee in toptee_split:
    os.system('mv %s.png ./resized_images/toptee/%s.png'%(toptee, toptee))